In [62]:
import pandas as pd
import numpy as np
from pandasql import sqldf
from sqlalchemy import create_engine
from datetime import datetime, timedelta
import calendar
import glob

In [99]:
#initial
csv_customer = './input/customer_info.csv'
csv_transaction_pattern = './input/product_*.csv'


In [101]:
#table customer
def tb_customer(csv_customer):
    customer_rd = pd.read_csv(csv_customer, skip_blank_lines=True)
    
    customer_rd = customer_rd.loc[:, ~customer_rd.columns.str.contains('^Unnamed')]
    
    filtered_customer = customer_rd[customer_rd['phone'].notnull()]
    
    return filtered_customer

In [105]:
# Fungsi untuk membaca semua data dari file CSV yang sesuai pola
def read_all_product_data(csv_transaction_pattern):
    key_columns = ['product_id', 'customer_id']
    
    # Menggunakan glob untuk mendapatkan semua file yang sesuai dengan pola
    all_files = glob.glob(csv_transaction_pattern)
    
    # Membaca dan menggabungkan semua file CSV ke dalam satu DataFrame
    all_data = pd.concat([pd.read_csv(file) for file in all_files], ignore_index=True)
    
    # Mengganti koma dengan spasi di kolom 'description'
    all_data['description'] = all_data['description'].str.replace(",", " ")
    
    # Mengubah format tanggal
    all_data['invoice_date'] = pd.to_datetime(all_data['invoice_date'], format='%m/%d/%Y %H:%M', errors='coerce')
    
    # Mengubah format tanggal menjadi yyyy-MM-dd
    all_data['invoice_date'] = all_data['invoice_date'].dt.strftime('%Y-%m-%d')
    
    # Menghapus baris dengan nilai NaN di kolom kunci
    clean_all_data = all_data.dropna(subset=key_columns).copy()
    
    # Mengubah tipe data 'customer_id' menjadi integer
    clean_all_data['customer_id'] = clean_all_data['customer_id'].astype(int)
    
    return clean_all_data



In [106]:
cleaning_transaction_data = read_all_product_data(csv_transaction_pattern)
cleaning_transaction_data

,product_id,invoice_no,stock_code,description,quantity,invoice_date,unit_price,customer_id,product_group,product_name,stock
1,TW210,540976,22900,SET 2 TEA TOWELS I LOVE LONDON,6,2024-01-12,2.95,12652,CLOTH,TOWELS,294
2,TW210,541877,22900,SET 2 TEA TOWELS I LOVE LONDON,6,2024-01-24,2.95,12559,CLOTH,TOWELS,288
3,PEN209,540365,22418,10 COLOUR SPACEBOY PEN,24,2024-01-06,0.85,12413,STATIONERY,PEN,2952
4,PEN209,540835,22418,10 COLOUR SPACEBOY PEN,24,2024-01-11,0.85,12724,STATIONERY,PEN,2928
5,PEN209,541138,22418,10 COLOUR SPACEBOY PEN,24,2024-01-14,0.85,12553,STATIONERY,PEN,2904
...,...,...,...,...,...,...,...,...,...,...,...
31161,PBG207,C580989,22331,WOODLAND PARTY BAG + STICKER SET,1,2024-12-06,1.65,12709,BAG,PARTY BAG,108.0
31162,PBG207,581570,22331,WOODLAND PARTY BAG + STICKER SET,8,2024-12-09,1.65,12662,BAG,PARTY BAG,100.0
31168,WW840,581171,84077,WORLD WAR 2 GLIDERS ASSTD DESIGNS,48,2024-12-07,0.29,12615,TOY,WORLD WAR ASSTD,76.0
31182,CK227,580152,16156S,WRAP PINK FAIRY CAKES,25,2024-12-02,0.42,12364,KITCHENWARE,CAKE CASE,175.0


In [115]:
#mapping data for table product
def product_table(clean_all_data):
    #key_columns = ['product_id', 'product_name', 'location']
    # Sort data by product_id
    sort_data_retail_cleanup = clean_all_data.sort_values(by=['product_id'], ascending=True)
    
    # Remove duplicates based on product_id
    dedup_data_retail_cleanup = sort_data_retail_cleanup.drop_duplicates(subset='product_name', keep='first').copy()  # Tambahkan .copy() di sini
    
    # Convert product_group to lowercase
    dedup_data_retail_cleanup.loc[:, 'product_group'] = dedup_data_retail_cleanup['product_group'].str.lower()  # Gunakan .loc[] di sini
    
    # Define location based on product_group
    location_map = {
        'accessories': 'Lt.2_Stand.1_accessories',
        'bag': 'Lt.2_Stand.4_bag',
        'box': 'Lt.1_Stand.1_box',
        'cloth': 'Lt.2_Stand.5_cloth',
        'electronic': 'Lt.1_Stand.2_electronic',
        'glassware': 'Lt.1_Stand.3_glassware',
        'kitchenware': 'Lt.1_Stand.4_kitchenware',
        'personal hygene': 'Lt.2_Stand.6_personal hygene',
        'stationery': 'Lt.2_Stand.2_stationery',
        'utensil': 'Lt.1_Stand.7_utensil',
        'tools': 'Lt.1_Stand.6_tools',
        'toy': 'Lt.2_Stand.3_toy'
    }
    
    # Map product_group ke location
    dedup_data_retail_cleanup.loc[:, 'location'] = dedup_data_retail_cleanup['product_group'].map(location_map).fillna('not register yet')  # Gunakan .loc[] di sini

    # Select the relevant columns
    #result_product = sort_data_retail_cleanup.drop_duplicates(subset = key_columns, keep='first')
    result_product = dedup_data_retail_cleanup[['product_name', 'location']].copy()
    result_product.loc[:, 'productID'] = range(1, len(result_product)+1)
    
    
    return result_product

In [116]:
data_product = product_table(cleaning_transaction_data)
data_product

,product_name,location,productID
21666,ALARM,Lt.2_Stand.1_accessories,1
31062,APRON,Lt.1_Stand.4_kitchenware,2
18625,ASSTD,Lt.2_Stand.1_accessories,3
26116,BOTTLE,Lt.1_Stand.3_glassware,4
12927,BREAD BIN CUTTER,Lt.1_Stand.7_utensil,5
...,...,...,...
21218,VARIOUS BAG,Lt.2_Stand.4_bag,78
7382,WASHBAG,Lt.2_Stand.4_bag,79
16027,CLOCK,Lt.2_Stand.1_accessories,80
145,CARD,Lt.2_Stand.1_accessories,81


In [98]:

cleaning_data_customer = tb_customer(csv_customer)

#cleaning_transaction_data = read_all_product_data(csv_transaction)

#data_product = product_table(cleaning_transaction_data)

In [67]:
cleaning_data_customer

,customer_id,customer_name,email_address,country,phone
0,12386,Allen Bowers,allen_bowers@example.com,Australia,+64 00 34
1,12388,Allison Huang,allison_huang@example.com,Australia,+64 00 35
2,12393,Alyssa Ford,alyssa_ford@example.com,Australia,+64 00 38
3,12415,Amy Campbell,amy_campbell@example.com,Australia,+64 00 56
4,12422,Amy Watson,amy_watson@example.com,Australia,+64 00 61
...,...,...,...,...,...
4363,18287,Aaron Edwards,aaron_edwards@example.com,United Kingdom,+44 00 1
4368,12558,Brittany Thornton,brittany_thornton@example.com,USA,+1 00 171
4369,12607,Charlotte Thomas,charlotte_thomas@example.com,USA,+1 00 214
4370,12646,Christopher Sanchez,christopher_sanchez@example.com,USA,+1 00 248


In [131]:
#join data
def join_data(data_customer, transaction_data):
    df_join_data = sqldf('''select 
                            tr.customer_id,
                            c.customer_name,
                            c.email_address as email,
                            c.country, c.phone,
                            tr.product_id,
                            tr.invoice_no,
                            tr.stock_code,
                            tr.description,
                            tr.invoice_date,
                            tr.quantity,
                            tr.unit_price as price,
                            tr.product_group,
                            tr.product_name,
                            tr.stock 
                            from cleaning_transaction_data tr
                            join cleaning_data_customer c on tr.customer_id = c.customer_id
                            where tr.quantity > 0''')
    
    return df_join_data

In [133]:
all_data_join = join_data(cleaning_data_customer, cleaning_transaction_data)
all_data_join.head()

,customer_id,customer_name,email,country,phone,product_id,invoice_no,stock_code,description,invoice_date,quantity,price,product_group,product_name,stock
0,12652,Claudia Jones,claudia_jones@example.com,France,+33 00 254,TW210,540976,22900,SET 2 TEA TOWELS I LOVE LONDON,2024-01-12,6,2.95,CLOTH,TOWELS,294
1,12559,Brittany Tran,brittany_tran@example.com,Germany,+49 00 172,TW210,541877,22900,SET 2 TEA TOWELS I LOVE LONDON,2024-01-24,6,2.95,CLOTH,TOWELS,288
2,12413,Amy Burke,amy_burke@example.com,France,+33 00 54,PEN209,540365,22418,10 COLOUR SPACEBOY PEN,2024-01-06,24,0.85,STATIONERY,PEN,2952
3,12724,David Garcia,david_garcia@example.com,France,+33 00 313,PEN209,540835,22418,10 COLOUR SPACEBOY PEN,2024-01-11,24,0.85,STATIONERY,PEN,2928
4,12553,Brianna Orr,brianna_orr@example.com,France,+33 00 168,PEN209,541138,22418,10 COLOUR SPACEBOY PEN,2024-01-14,24,0.85,STATIONERY,PEN,2904


In [138]:
#create table orders
def tb_orders(all_data_join):
    amount_per_unit = (all_data_join['quantity'] * all_data_join['price']).round(2)
    
    #menambahkan amount ke Dataframe
    all_data_join['amount_per_unit'] = amount_per_unit
    
    orders = all_data_join[['customer_id','invoice_date', 'invoice_no', 'amount_per_unit']]
    
    # Mengelompokkan data berdasarkan 'invoice_number' dan 'invoice_date'
    # Lalu menjumlahkan 'amount_per_unit' untuk setiap grup
    total_amount_per_invoice = orders.groupby(['customer_id','invoice_no', 'invoice_date'])['amount_per_unit'].sum().reset_index()
    
    total_amount_per_invoice.loc[:, 'order_id'] = range(1, len(total_amount_per_invoice)+1)
    
    return total_amount_per_invoice

In [139]:
df_order = tb_orders(all_data_join)
df_order

,customer_id,invoice_no,invoice_date,amount_per_unit,order_id
0,12348,541998,2024-01-25,187.44,1
1,12348,548955,2024-04-05,120.00,2
2,12348,568172,2024-09-25,120.00,3
3,12353,553900,2024-05-19,89.00,4
4,12355,552449,2024-05-09,234.40,5
...,...,...,...,...,...
1452,17443,551736,2024-05-04,534.24,1453
1453,17444,557322,2024-06-20,766.06,1454
1454,17444,559557,2024-07-11,689.71,1455
1455,17508,545579,2024-03-04,133.75,1456
